## Data Preprocessing

In [1]:
import sys
sys.path.insert(0, '..')
import os

import numpy as np
import pandas as pd

In [2]:
import zipfile
with zipfile.ZipFile('./data/labeledTrainData.tsv.zip', 'r') as z:
    z.extractall('./data/')

In [3]:
data_train = pd.read_csv('./data/labeledTrainData.tsv', sep='\t')

In [4]:
print(data_train.shape)
data_train.head(5)

(25000, 3)


,id,sentiment,review
0,5814_8,1,With all this stuff going down at the moment w...
1,2381_9,1,"\The Classic War of the Worlds\"" by Timothy Hi..."
2,7759_3,0,The film starts with a manager (Nicholas Bell)...
3,3630_4,0,It must be assumed that those who praised this...
4,9495_8,1,Superbly trashy and wondrously unpretentious 8...


In [5]:
data_train['review'][0]

"With all this stuff going down at the moment with MJ i've started listening to his music, watching the odd documentary here and there, watched The Wiz and watched Moonwalker again. Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent. Moonwalker is part biography, part feature film which i remember going to see at the cinema when it was originally released. Some of it has subtle messages about MJ's feeling towards the press and also the obvious message of drugs are bad m'kay.<br /><br />Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring. Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him.<br /><br />The actual feature film bit when it finally sta

In [6]:
data_train.review.shape[0]

25000

In [7]:
from bs4 import BeautifulSoup
import re

def clean_str(string):
    """
    Tokenization/string cleaning for all datasets 
    Every dataset is lower cased
    """
    string = re.sub(r"[^A-Za-z0-9(),!?\'\`]", " ", string)     
    string = re.sub(r"\'s", " \'s", string) 
    string = re.sub(r"\'ve", " \'ve", string) 
    string = re.sub(r"n\'t", " n\'t", string) 
    string = re.sub(r"\'re", " \'re", string) 
    string = re.sub(r"\'d", " \'d", string) 
    string = re.sub(r"\'ll", " \'ll", string) 
    string = re.sub(r",", " , ", string) 
    string = re.sub(r"!", " ! ", string) 
    string = re.sub(r"\(", " \( ", string) 
    string = re.sub(r"\)", " \) ", string) 
    string = re.sub(r"\?", " \? ", string) 
    string = re.sub(r"\s{2,}", " ", string)    
    return string.strip().lower()

texts = []
labels = []
for idx in range(data_train.review.shape[0]):
    text = BeautifulSoup(data_train.review[idx], "lxml")
    texts.append(clean_str(text.get_text()))
    labels.append(data_train.sentiment[idx])

In [8]:
len(texts)

25000

In [9]:
texts[0]

"with all this stuff going down at the moment with mj i 've started listening to his music , watching the odd documentary here and there , watched the wiz and watched moonwalker again maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent moonwalker is part biography , part feature film which i remember going to see at the cinema when it was originally released some of it has subtle messages about mj 's feeling towards the press and also the obvious message of drugs are bad m'kay visually impressive but of course this is all about michael jackson so unless you remotely like mj in anyway then you are going to hate this and find it boring some may call mj an egotist for consenting to the making of this movie but mj and most of his fans would say that he made it for the fans which if true is really nice of him the actual feature film bit when it finally starts is only on for 20 m

In [21]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical

from keras.layers import Embedding
from keras.layers import Dense, Input, Flatten, concatenate, TimeDistributed
from keras.layers import Conv1D, MaxPooling1D, Dropout, Concatenate, SpatialDropout1D, GlobalAveragePooling1D, GlobalMaxPooling1D
from keras.layers import Bidirectional, LSTM, GRU, CuDNNLSTM, CuDNNGRU
from keras.models import Model

from keras import backend as K
from keras.engine.topology import Layer, InputSpec
from keras import initializers

MAX_SEQUENCE_LENGTH = 1000
MAX_NB_WORDS = 20000
EMBEDDING_DIM = 100
VALIDATION_SPLIT = 0.2

In [11]:
tokenizer = Tokenizer(nb_words=MAX_NB_WORDS)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)

word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

C:\Users\sun\Anaconda3\lib\site-packages\keras_preprocessing\text.py:175: UserWarning: The `nb_words` argument in `Tokenizer` has been renamed `num_words`.
  warnings.warn('The `nb_words` argument in `Tokenizer` '


Found 81988 unique tokens.


In [12]:
data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)
labels = to_categorical(np.asarray(labels))
print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', labels.shape)

Shape of data tensor: (25000, 1000)
Shape of label tensor: (25000, 2)


In [13]:
indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]
nb_validation_samples = int(VALIDATION_SPLIT * data.shape[0])

x_train = data[:-nb_validation_samples]
y_train = labels[:-nb_validation_samples]
x_val = data[-nb_validation_samples:]
y_val = labels[-nb_validation_samples:]

print('Number of positive and negative reviews in traing and validation set ')
print(y_train.sum(axis=0))
print(y_val.sum(axis=0))

Number of positive and negative reviews in traing and validation set 
[10073.  9927.]
[2427. 2573.]


In [14]:
GLOVE_DIR = "../pre_trained word embeddings/glove.6B"
embeddings_index = {}
f = open(os.path.join(GLOVE_DIR, 'glove.6B.100d.txt'), encoding='utf-8')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()
print('Total %s word vectors in Glove 6B 100d.' % len(embeddings_index))

Total 400000 word vectors in Glove 6B 100d.


In [15]:
embedding_matrix = np.random.random((len(word_index) + 1, EMBEDDING_DIM))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

In [16]:
embedding_layer = Embedding(len(word_index) + 1,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=True)

## Deep Model

### TextCNN

In [21]:
from keras.models import Sequential
from keras.layers import Convolution1D, Dropout, Activation 

NB_FILTER = 256
FILTER_LENGTH = 5

print ("Training model.")
model = Sequential()
model.add(embedding_layer)
model.add(Convolution1D(nb_filter=NB_FILTER,
                        filter_length=FILTER_LENGTH,
                        border_mode='valid',
                        activation='relu',
                        subsample_length=1))
# use max pooling:
model.add(MaxPooling1D(pool_length=model.output_shape[1]))
# We flatten the output of the conv layer,
# so that we can add a vanilla dense layer:
model.add(Flatten())
# We add a vanilla hidden layer:
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Activation('relu'))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.2))
model.add(Activation('relu'))
# We project onto a single unit output layer, and squash it with a sigmoid:
model.add(Dense(2, activation='sigmoid'))

model.summary() 

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['acc'])

model.fit(x_train, y_train, validation_data=(x_val, y_val),nb_epoch=10, batch_size=128)

Training model.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 1000, 100)         8198900   
_________________________________________________________________
conv1d_4 (Conv1D)            (None, 996, 256)          128256    
_________________________________________________________________
max_pooling1d_3 (MaxPooling1 (None, 1, 256)            0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 256)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 256)               65792     
_________________________________________________________________
dropout_5 (Dropout)          (None, 256)               0         
_________________________________________________________________
activation_5 (Activation)    (None, 256)               0    

C:\Users\sun\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(activation="relu", filters=256, kernel_size=5, strides=1, padding="valid")`
  from ipykernel import kernelapp as app
C:\Users\sun\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: UserWarning: Update your `MaxPooling1D` call to the Keras 2 API: `MaxPooling1D(pool_size=996)`
C:\Users\sun\Anaconda3\lib\site-packages\ipykernel_launcher.py:37: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


Train on 20000 samples, validate on 5000 samples
Epoch 1/10
20000/20000 [==============================] - 23s 1ms/step - loss: 0.3280 - acc: 0.8358 - val_loss: 0.3656 - val_acc: 0.8817
Epoch 2/10
20000/20000 [==============================] - 22s 1ms/step - loss: 0.0751 - acc: 0.9739 - val_loss: 0.4081 - val_acc: 0.8796
Epoch 3/10
20000/20000 [==============================] - 22s 1ms/step - loss: 0.0251 - acc: 0.9913 - val_loss: 0.6895 - val_acc: 0.8639
Epoch 4/10
20000/20000 [==============================] - 22s 1ms/step - loss: 0.0097 - acc: 0.9970 - val_loss: 0.7501 - val_acc: 0.8730
Epoch 5/10
20000/20000 [==============================] - 22s 1ms/step - loss: 0.0130 - acc: 0.9955 - val_loss: 0.7973 - val_acc: 0.8491
Epoch 6/10
20000/20000 [==============================] - 22s 1ms/step - loss: 0.0088 - acc: 0.9970 - val_loss: 0.8852 - val_acc: 0.8798
Epoch 7/10
20000/20000 [==============================] - 22s 1ms/step - loss: 0.0039 - acc: 0.9988 - val_loss: 0.8906 - val_acc:

In [31]:
# applying a more complex convolutional approach
convs = []
filter_sizes = [3,4,5]

sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
embedded_sequences = embedding_layer(sequence_input)

for fsz in filter_sizes:
    l_conv = Conv1D(nb_filter=128,filter_length=fsz,activation='relu')(embedded_sequences)
    l_pool = MaxPooling1D(pool_length=MAX_SEQUENCE_LENGTH-fsz+1)(l_conv)
    convs.append(l_pool)
l_merge = Concatenate(axis=1)(convs) 
l_flat = Flatten()(l_merge)
l_dense1= Dense(256, activation='relu')(l_flat)
l_dropout1 = Dropout(0.5)(l_dense1)
l_dense = Dense(128, activation='relu')(l_dropout1)
l_dropout2 = Dropout(0.3)(l_dense)
preds = Dense(2, activation='softmax')(l_dropout2)
model = Model(sequence_input, preds)

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['acc'])

model.summary() 

model.fit(x_train, y_train, validation_data=(x_val, y_val),nb_epoch=10, batch_size=128)

C:\Users\sun\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(activation="relu", filters=128, kernel_size=3)`
  if __name__ == '__main__':
C:\Users\sun\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: UserWarning: Update your `MaxPooling1D` call to the Keras 2 API: `MaxPooling1D(pool_size=998)`
  # Remove the CWD from sys.path while we load stuff.
C:\Users\sun\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(activation="relu", filters=128, kernel_size=4)`
  if __name__ == '__main__':
C:\Users\sun\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: UserWarning: Update your `MaxPooling1D` call to the Keras 2 API: `MaxPooling1D(pool_size=997)`
  # Remove the CWD from sys.path while we load stuff.
C:\Users\sun\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(activation="rel

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_8 (InputLayer)            (None, 1000)         0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 1000, 100)    8198900     input_8[0][0]                    
__________________________________________________________________________________________________
conv1d_25 (Conv1D)              (None, 998, 128)     38528       embedding_1[11][0]               
__________________________________________________________________________________________________
conv1d_26 (Conv1D)              (None, 997, 128)     51328       embedding_1[11][0]               
__________________________________________________________________________________________________
conv1d_27 

In [24]:
sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
embedded_sequences = embedding_layer(sequence_input)
l_cov1= Conv1D(128, 5, activation='relu')(embedded_sequences)
l_pool1 = MaxPooling1D(5)(l_cov1)
l_cov2 = Conv1D(128, 5, activation='relu')(l_pool1)
l_pool2 = MaxPooling1D(5)(l_cov2)
l_cov3 = Conv1D(128, 5, activation='relu')(l_pool2)
l_pool3 = MaxPooling1D(35)(l_cov3)  # global max pooling
l_flat = Flatten()(l_pool3)
l_dropout1 = Dropout(0.5)(l_flat)
l_dense = Dense(128, activation='relu')(l_dropout1)
l_dropout2 = Dropout(0.3)(l_dense)
preds = Dense(2, activation='softmax')(l_dropout2)

model = Model(sequence_input, preds)
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['acc'])
print("model fitting - simplified convolutional neural network")
model.summary()

model fitting - simplified convolutional neural network
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 1000)              0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 1000, 100)         8198900   
_________________________________________________________________
conv1d_7 (Conv1D)            (None, 996, 128)          64128     
_________________________________________________________________
max_pooling1d_5 (MaxPooling1 (None, 199, 128)          0         
_________________________________________________________________
conv1d_8 (Conv1D)            (None, 195, 128)          82048     
_________________________________________________________________
max_pooling1d_6 (MaxPooling1 (None, 39, 128)           0         
_________________________________________________________________
conv1d_9 (Conv1D)   

In [25]:
model.fit(x_train, y_train, validation_data=(x_val, y_val),
          nb_epoch=5, batch_size=128)

C:\Users\sun\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  


Train on 20000 samples, validate on 5000 samples
Epoch 1/5
20000/20000 [==============================] - 11s 554us/step - loss: 0.4344 - acc: 0.7590 - val_loss: 0.3536 - val_acc: 0.8756
Epoch 2/5
20000/20000 [==============================] - 10s 483us/step - loss: 0.1241 - acc: 0.9559 - val_loss: 0.4720 - val_acc: 0.8718
Epoch 3/5
20000/20000 [==============================] - 10s 482us/step - loss: 0.0504 - acc: 0.9832 - val_loss: 0.6448 - val_acc: 0.8688
Epoch 4/5
20000/20000 [==============================] - 10s 485us/step - loss: 0.0323 - acc: 0.9896 - val_loss: 0.7590 - val_acc: 0.8674
Epoch 5/5
20000/20000 [==============================] - 10s 485us/step - loss: 0.0161 - acc: 0.9945 - val_loss: 0.8486 - val_acc: 0.8684


In [26]:
# applying a more complex convolutional approach
convs = []
filter_sizes = [3,4,5]

sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
embedded_sequences = embedding_layer(sequence_input)

for fsz in filter_sizes:
    l_conv = Conv1D(nb_filter=128,filter_length=fsz,activation='relu')(embedded_sequences)
    l_pool = MaxPooling1D(5)(l_conv)
    convs.append(l_pool)
l_merge = Concatenate(axis=1)(convs)    
# l_merge = Merge(mode='concat', concat_axis=1)(convs)
l_cov1= Conv1D(128, 5, activation='relu')(l_merge)
l_pool1 = MaxPooling1D(5)(l_cov1)
l_cov2 = Conv1D(128, 5, activation='relu')(l_pool1)
l_pool2 = MaxPooling1D(30)(l_cov2)
l_flat = Flatten()(l_pool2)
l_dense = Dense(128, activation='relu')(l_flat)
preds = Dense(2, activation='softmax')(l_dense)

model = Model(sequence_input, preds)
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['acc'])

print("model fitting - more complex convolutional neural network")
model.summary()
model.fit(x_train, y_train, validation_data=(x_val, y_val),
          nb_epoch=10, batch_size=50)

C:\Users\sun\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(activation="relu", filters=128, kernel_size=3)`
  if __name__ == '__main__':
C:\Users\sun\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(activation="relu", filters=128, kernel_size=4)`
  if __name__ == '__main__':
C:\Users\sun\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(activation="relu", filters=128, kernel_size=5)`
  if __name__ == '__main__':
C:\Users\sun\Anaconda3\lib\site-packages\ipykernel_launcher.py:30: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


model fitting - more complex convolutional neural network
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 1000)         0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 1000, 100)    8198900     input_3[0][0]                    
__________________________________________________________________________________________________
conv1d_10 (Conv1D)              (None, 998, 128)     38528       embedding_1[6][0]                
__________________________________________________________________________________________________
conv1d_11 (Conv1D)              (None, 997, 128)     51328       embedding_1[6][0]                
___________________________________________________

### Bi-LSTM

In [32]:
sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
embedded_sequences = embedding_layer(sequence_input)
l_lstm = Bidirectional(CuDNNLSTM(100))(embedded_sequences)
preds = Dense(2, activation='softmax')(l_lstm)
model = Model(sequence_input, preds)
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['acc'])

print("model fitting - Bidirectional LSTM")
model.summary()
model.fit(x_train, y_train, validation_data=(x_val, y_val),
          nb_epoch=10, batch_size=50)

model fitting - Bidirectional LSTM
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_9 (InputLayer)         (None, 1000)              0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 1000, 100)         8198900   
_________________________________________________________________
bidirectional_1 (Bidirection (None, 200)               161600    
_________________________________________________________________
dense_21 (Dense)             (None, 2)                 402       
Total params: 8,360,902
Trainable params: 8,360,902
Non-trainable params: 0
_________________________________________________________________


C:\Users\sun\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  del sys.path[0]


Train on 20000 samples, validate on 5000 samples
Epoch 1/10
20000/20000 [==============================] - 50s 2ms/step - loss: 0.3962 - acc: 0.8282 - val_loss: 0.3171 - val_acc: 0.8844
Epoch 2/10
20000/20000 [==============================] - 49s 2ms/step - loss: 0.1533 - acc: 0.9480 - val_loss: 0.3891 - val_acc: 0.8710
Epoch 3/10
20000/20000 [==============================] - 49s 2ms/step - loss: 0.0722 - acc: 0.9763 - val_loss: 0.3632 - val_acc: 0.8862
Epoch 4/10
20000/20000 [==============================] - 49s 2ms/step - loss: 0.0394 - acc: 0.9868 - val_loss: 0.5082 - val_acc: 0.8852
Epoch 5/10
20000/20000 [==============================] - 48s 2ms/step - loss: 0.0278 - acc: 0.9913 - val_loss: 0.5018 - val_acc: 0.8818
Epoch 6/10
20000/20000 [==============================] - 49s 2ms/step - loss: 0.0162 - acc: 0.9950 - val_loss: 0.5982 - val_acc: 0.8870
Epoch 7/10
20000/20000 [==============================] - 48s 2ms/step - loss: 0.0110 - acc: 0.9965 - val_loss: 0.6488 - val_acc:

### RCNN

In [23]:
sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
embedded_sequences = embedding_layer(sequence_input)
l_dropout = SpatialDropout1D(0.3)(embedded_sequences)
l_gru = Bidirectional(CuDNNGRU(100, return_sequences=True))(l_dropout)
l_cov = Conv1D(64, kernel_size = 2, padding = "valid", kernel_initializer = "he_uniform")(l_gru)
avg_pool = GlobalAveragePooling1D()(l_cov)
max_pool = GlobalMaxPooling1D()(l_cov)
l_conc = concatenate([avg_pool, max_pool])
preds = Dense(2, activation='softmax')(l_conc)
model = Model(sequence_input, preds)
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['acc'])

print("model fitting - RCNN")
model.summary()
model.fit(x_train, y_train, validation_data=(x_val, y_val),
          nb_epoch=10, batch_size=50)

model fitting - RCNN
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            (None, 1000)         0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 1000, 100)    8198900     input_4[0][0]                    
__________________________________________________________________________________________________
spatial_dropout1d_3 (SpatialDro (None, 1000, 100)    0           embedding_1[3][0]                
__________________________________________________________________________________________________
bidirectional_3 (Bidirectional) (None, 1000, 200)    121200      spatial_dropout1d_3[0][0]        
________________________________________________________________________________________

C:\Users\sun\Anaconda3\lib\site-packages\ipykernel_launcher.py:18: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


Train on 20000 samples, validate on 5000 samples
Epoch 1/10
20000/20000 [==============================] - 57s 3ms/step - loss: 0.4715 - acc: 0.7631 - val_loss: 0.3183 - val_acc: 0.8660
Epoch 2/10
20000/20000 [==============================] - 54s 3ms/step - loss: 0.3096 - acc: 0.8705 - val_loss: 0.2766 - val_acc: 0.8812
Epoch 3/10
20000/20000 [==============================] - 53s 3ms/step - loss: 0.2381 - acc: 0.9021 - val_loss: 0.2530 - val_acc: 0.8938
Epoch 4/10
20000/20000 [==============================] - 53s 3ms/step - loss: 0.1834 - acc: 0.9275 - val_loss: 0.2509 - val_acc: 0.9004
Epoch 5/10
20000/20000 [==============================] - 55s 3ms/step - loss: 0.1455 - acc: 0.9439 - val_loss: 0.3003 - val_acc: 0.8926
Epoch 6/10
20000/20000 [==============================] - 56s 3ms/step - loss: 0.1039 - acc: 0.9620 - val_loss: 0.2585 - val_acc: 0.9106
Epoch 7/10
20000/20000 [==============================] - 55s 3ms/step - loss: 0.0770 - acc: 0.9723 - val_loss: 0.3142 - val_acc:

### HRNN

In [17]:
from keras.preprocessing.text import Tokenizer, text_to_word_sequence
from nltk import tokenize

MAX_SENT_LENGTH = 100
MAX_SENTS = 15

reviews = []
labels = []
texts = []

for idx in range(data_train.review.shape[0]):
    text = BeautifulSoup(data_train.review[idx], "lxml")
    text = clean_str(text.get_text())
    texts.append(text)
    sentences = tokenize.sent_tokenize(text)
    reviews.append(sentences)
    labels.append(data_train.sentiment[idx])

data = np.zeros((len(texts), MAX_SENTS, MAX_SENT_LENGTH), dtype='int32')

for i, sentences in enumerate(reviews):
    for j, sent in enumerate(sentences):
        if j < MAX_SENTS:
            wordTokens = text_to_word_sequence(sent)
            k = 0
            for _, word in enumerate(wordTokens):
                if k < MAX_SENT_LENGTH and tokenizer.word_index[word] < MAX_NB_WORDS:
                    data[i, j, k] = tokenizer.word_index[word]
                    k = k + 1

word_index = tokenizer.word_index
print('Total %s unique tokens.' % len(word_index))

labels = to_categorical(np.asarray(labels))
print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', labels.shape)

indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]
nb_validation_samples = int(VALIDATION_SPLIT * data.shape[0])

x_train = data[:-nb_validation_samples]
y_train = labels[:-nb_validation_samples]
x_val = data[-nb_validation_samples:]
y_val = labels[-nb_validation_samples:]

print('Number of positive and negative reviews in traing and validation set')
print(y_train.sum(axis=0))
print(y_val.sum(axis=0))

Total 81988 unique tokens.
Shape of data tensor: (25000, 15, 100)
Shape of label tensor: (25000, 2)
Number of positive and negative reviews in traing and validation set
[ 9998. 10002.]
[2502. 2498.]


In [18]:
embedding_layer = Embedding(len(word_index) + 1,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=MAX_SENT_LENGTH,
                            trainable=True,
                            mask_zero=True)

In [22]:
sentence_input = Input(shape=(MAX_SENT_LENGTH,), dtype='int32')
embedded_sequences = embedding_layer(sentence_input)
l_lstm = Bidirectional(GRU(100))(embedded_sequences)
sentEncoder = Model(sentence_input, l_lstm)

review_input = Input(shape=(MAX_SENTS,MAX_SENT_LENGTH), dtype='int32')
review_encoder = TimeDistributed(sentEncoder)(review_input)
l_lstm_sent = Bidirectional(GRU(100))(review_encoder)
preds = Dense(2, activation='softmax')(l_lstm_sent)
model = Model(review_input, preds)

model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['acc'])
model.summary()
print("model fitting - Hierachical attention network")
model.fit(x_train, y_train, validation_data=(x_val, y_val),
          nb_epoch=10, batch_size=50)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         (None, 15, 100)           0         
_________________________________________________________________
time_distributed_1 (TimeDist (None, 15, 200)           8319500   
_________________________________________________________________
bidirectional_4 (Bidirection (None, 200)               180600    
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 402       
Total params: 8,500,502
Trainable params: 8,500,502
Non-trainable params: 0
_________________________________________________________________
model fitting - Hierachical attention network


C:\Users\sun\Anaconda3\lib\site-packages\ipykernel_launcher.py:18: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


Train on 20000 samples, validate on 5000 samples
Epoch 1/10
20000/20000 [==============================] - 125s 6ms/step - loss: 0.5248 - acc: 0.7323 - val_loss: 0.4053 - val_acc: 0.8270
Epoch 2/10
20000/20000 [==============================] - 121s 6ms/step - loss: 0.3296 - acc: 0.8594 - val_loss: 0.4344 - val_acc: 0.8120
Epoch 3/10
20000/20000 [==============================] - 122s 6ms/step - loss: 0.2492 - acc: 0.8996 - val_loss: 0.3192 - val_acc: 0.8744
Epoch 4/10
20000/20000 [==============================] - 122s 6ms/step - loss: 0.1835 - acc: 0.9298 - val_loss: 0.3468 - val_acc: 0.8696
Epoch 5/10
20000/20000 [==============================] - 122s 6ms/step - loss: 0.1294 - acc: 0.9527 - val_loss: 0.3730 - val_acc: 0.8686
Epoch 6/10
20000/20000 [==============================] - 124s 6ms/step - loss: 0.0801 - acc: 0.9729 - val_loss: 0.4922 - val_acc: 0.8670
Epoch 7/10
20000/20000 [==============================] - 124s 6ms/step - loss: 0.0423 - acc: 0.9864 - val_loss: 0.5912 - v

### char-models

> https://offbit.github.io/how-to-read/

> https://github.com/offbit/char-models

> https://github.com/keras-team/keras/blob/master/examples/lstm_text_generation.py

In [1]:
import pandas as pd
from keras.models import Model
from keras.layers import Dense, Input, Dropout, MaxPooling1D, Conv1D
from keras.layers import LSTM, Lambda
from keras.layers import TimeDistributed, Bidirectional
from keras.layers.normalization import BatchNormalization
import numpy as np
import tensorflow as tf
import re
import keras.callbacks
import sys
import os


def binarize(x, sz=71):
    return tf.to_float(tf.one_hot(x, sz, on_value=1, off_value=0, axis=-1))


def binarize_outshape(in_shape):
    return in_shape[0], in_shape[1], 71


def striphtml(html):
    p = re.compile(r'<.*?>')
    return p.sub('', html)


def clean(s):
    return re.sub(r'[^\x00-\x7f]', r'', s)


data = pd.read_csv("./data/labeledTrainData.tsv", header=0, delimiter="\t", quoting=3)
txt = ''
docs = []
sentences = []
sentiments = []

for cont, sentiment in zip(data.review, data.sentiment):
    sentences = re.split(r'(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?)\s', clean(striphtml(cont)))
    sentences = [sent.lower() for sent in sentences]
    docs.append(sentences)
    sentiments.append(sentiment)

num_sent = []
for doc in docs:
    num_sent.append(len(doc))
    for s in doc:
        txt += s

chars = set(txt)

print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

print('Sample doc{}'.format(docs[1200]))

maxlen = 512
max_sentences = 15

X = np.ones((len(docs), max_sentences, maxlen), dtype=np.int64) * -1
y = np.array(sentiments)

for i, doc in enumerate(docs):
    for j, sentence in enumerate(doc):
        if j < max_sentences:
            for t, char in enumerate(sentence[-maxlen:]):
                X[i, j, (maxlen - 1 - t)] = char_indices[char]

print('Sample chars in X:{}'.format(X[1200, 2]))
print('y:{}'.format(y[1200]))

ids = np.arange(len(X))
np.random.shuffle(ids)

# shuffle
X = X[ids]
y = y[ids]

X_train = X[:20000]
X_test = X[20000:]

y_train = y[:20000]
y_test = y[20000:]

filter_length = [5, 3, 3]
nb_filter = [196, 196, 256]
pool_length = 2
# document input
document = Input(shape=(max_sentences, maxlen), dtype='int64')
# sentence input
in_sentence = Input(shape=(maxlen,), dtype='int64')
# char indices to one hot matrix, 1D sequence to 2D 
embedded = Lambda(binarize, output_shape=binarize_outshape)(in_sentence)
# embedded: encodes sentence
for i in range(len(nb_filter)):
    embedded = Conv1D(filters=nb_filter[i],
                      kernel_size=filter_length[i],
                      padding='valid',
                      activation='relu',
                      kernel_initializer='glorot_normal',
                      strides=1)(embedded)

    embedded = Dropout(0.1)(embedded)
    embedded = MaxPooling1D(pool_size=pool_length)(embedded)

bi_lstm_sent = \
    Bidirectional(LSTM(128, return_sequences=False, dropout=0.15, recurrent_dropout=0.15, implementation=0))(embedded)

# sent_encode = merge([forward_sent, backward_sent], mode='concat', concat_axis=-1)
sent_encode = Dropout(0.3)(bi_lstm_sent)
# sentence encoder
encoder = Model(inputs=in_sentence, outputs=sent_encode)
encoder.summary()

encoded = TimeDistributed(encoder)(document)
# encoded: sentences to bi-lstm for document encoding 
b_lstm_doc = \
    Bidirectional(LSTM(128, return_sequences=False, dropout=0.15, recurrent_dropout=0.15, implementation=0))(encoded)

output = Dropout(0.3)(b_lstm_doc)
output = Dense(128, activation='relu')(output)
output = Dropout(0.3)(output)
output = Dense(1, activation='sigmoid')(output)

model = Model(inputs=document, outputs=output)

model.summary()

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(X_train, y_train, validation_data=(X_test, y_test), batch_size=16,
          epochs=15, shuffle=True)

C:\Users\sun\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


total chars: 71
Sample doc['"the premise is amazing and the some of the acting, notably sally kellerman and anthony rapp, is charming...', 'but this film is near unwatchable.', 'the music sounds as if it comes from some sort of the royalty free online site and the lyrics as if they were written with a rhyming dictionary open on the lap.', 'most of the singing is off-key.', 'i think they may have filmed with the singing accapella and put in the music under it...', 'the dialogue is really stupid and trite.', 'the movie works best when it is actually talking about the real estate but unfortunately it strays to often into stupid farcical sub-plots.', 'i found myself checking my watch after ther first twenty minutes and after 40 wondering \'when is it ever going to end.\'"']
Sample chars in X:[-1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 

C:\Users\sun\Anaconda3\lib\site-packages\keras\layers\recurrent.py:2068: UserWarning: `implementation=0` has been deprecated, and now defaults to `implementation=1`.Please update your layer call.
  warnings.warn('`implementation=0` has been deprecated, '


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 512)               0         
_________________________________________________________________
lambda_1 (Lambda)            (None, 512, 71)           0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 508, 196)          69776     
_________________________________________________________________
dropout_1 (Dropout)          (None, 508, 196)          0         
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 254, 196)          0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 252, 196)          115444    
_________________________________________________________________
dropout_2 (Dropout)          (None, 252, 196)          0         
__________